In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Imports

In [1]:
import os
import pandas as pd
import tqdm
import re
import torch
import json
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf
from preprocess import *
from vocabulary import *
from model import *
from model_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\swaga\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Pre Processing Dataset

In [ ]:
# train_df = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_train.csv')
# val_df   = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_dev.csv')
# # train_df = pd.read_csv('../datasets/snli/snli_/snli_1.0_train.csv')
# # val_df   = pd.read_csv('../datasets/snli/snli_/snli_1.0_dev.csv')
# train_df, val_df = preprocess_text(train_df, val_df)
# print(len(train_df))
# print(len(val_df))

In [ ]:
# len(train_df['sentence2'][8])

In [ ]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
rw = pd.read_csv('test_sets/ins.csv', encoding='ansi')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw.drop(['original_summary'], axis = 1, inplace=True)
rw.rename(columns = {'text':'sentence1'}, inplace = True)
rw.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)

# print(rw)
gold = [-1] * 100
rw.insert(loc=0, column='gold_label', value=gold)
rw = rw[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw = rw.dropna()
rw.head()

In [ ]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
rw_ = pd.read_csv('test_sets/ins.csv', encoding='ansi')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw_.drop(['text'], axis = 1, inplace=True)
rw_.rename(columns = {'original_summary':'sentence1'}, inplace = True)
rw_.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)

gold = [-1] * 100
rw_.insert(loc=0, column='gold_label', value=gold)
rw_ = rw_[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw_ = rw_.dropna()
rw_.head()

In [ ]:
rw_.head()

In [ ]:
rw['sentence1'][0]

In [ ]:
rw_['sentence2'][0]

In [ ]:
len(rw['sentence2'][8])

In [ ]:
# train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
# train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
# val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
# val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw['sentence1'] = rw['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw['sentence2'] = rw['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw_['sentence1'] = rw_['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw_['sentence2'] = rw_['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [ ]:
# train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
# val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
# print(train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)])
# print(val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)])

In [ ]:
# train_df.head()

In [ ]:
# val_df.head()

In [ ]:
# train_val_df = pd.concat([train_df, val_df])

In [ ]:
# sentence_pairs, _ = pair_generator(train_val_df)
rw_sentence_pairs, __ = pair_generator(rw)
rw_sentence_pairs_, __ = pair_generator(rw_)
# train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
# val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

# labels = set(train_sentence_labels)
# print(labels)

# tag2idx = {word: i for i, word in enumerate(labels)}
# print(tag2idx)
tag2idx = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
print(tag2idx)

# train_labels = [tag2idx[t] for t in train_sentence_labels]
# val_labels = [tag2idx[t] for t in val_sentence_labels]

In [ ]:
rw_sentence_pairs_[0]

In [ ]:
# vocab = Vocabulary()

# for data in [rw_sentence_pairs]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)
    
# for data in [rw_sentence_pairs_]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# for data in [sentence_pairs]:
#   for sentence_pair in data:
#     premise    = sentence_pair[0]
#     hypothesis = sentence_pair[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# with open('saved_vocabs/voc.class', 'wb') as vocab_file:
#     pickle.dump(vocab, vocab_file)
# print("Vocab size:", len(vocab.word2index))

In [ ]:
with open('saved_vocabs/voc.class', 'rb') as vocab_file_r:
    vocab = pickle.load(vocab_file_r)
print("Vocab size:", len(vocab.word2index))

In [ ]:
index2word = {}
for wrd, idx in vocab.word2index.items():
    # print(wrd, idx)
    index2word[idx] = wrd

## Model

In [ ]:
EPOCHS = 15
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 128
LEARNING_RATE = 0.005
STACKED_LAYERS = 2
EMBEDDING_PATH = '../../../../embeddings/google_news/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '../../../../embeddings/glove/glove.6B.300d.txt'
# EMBEDDING_PATH = '../embeddings/google_news/GoogleNews-vectors-negative300.bin'
# GLOVE_EMBEDDING = '../embeddings/glove/glove.6B.300d.txt'

initiate_model_vocab(vocab, tag2idx)

In [ ]:
train_data = DataSetLoader(get_pair_indices(vocab, train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(vocab, val_sentence_pairs), val_labels)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

In [ ]:
embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [ ]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [ ]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in lstm_model.state_dict():
    print(param_tensor, "\t", lstm_model.state_dict()[param_tensor].size())

In [ ]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(lstm_model, 'sm_2')

In [3]:
lstm_model = torch.load('saved_models/sm_2')
lstm_model.eval()

LSTM(
  (embedding): Embedding(33589, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=512, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [4]:
print(lstm_model.output)

Linear(in_features=32, out_features=3, bias=True)


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f'runs/SNLI/tensorboard_test')

In [ ]:
sentence_o = [''] * len(rw)
sentence_p = [''] * len(rw)
# sentence_o = ['']
# sentence_p = ['']

# sentence_o[0] = "rock music is very good for public"
# sentence_p[0] = "people dont like rock and roll"
# sentence_o = "uk india business council chairperson said 60 british firms likely increase investment india next years firms positive reforms fdi introduced last two years added india even important britain said"
# sentence_p = "60 british firms to increase investment in india"
# sentence_o = clean_text(sentence_o)
# sentence_p = clean_text(sentence_p)


# sen_p1 = [(sentence_o, sentence_p)]
# # sen_p2 = []
# for i in range(rw):
#   sen1 = sentence_o[i]
#   sen2 = sentence_p[i]
#   sen_p1.append((sen1, sen2))
# for i in range(rw_):
#   sen2 = sentence_o[i]
#   sen1 = sentence_p[i]
#   sen_p2.append((sen1, sen2))

# sen_p1

In [ ]:
rrw_sentence_pairs_ = []
for i in range(len(rw_sentence_pairs_)):
    rrw_sentence_pairs_.append((rw_sentence_pairs_[i][1], rw_sentence_pairs_[i][0]))
print('rw[0]: ', rw_sentence_pairs_[0])
print('rrw[0]: ', rrw_sentence_pairs_[0])

In [ ]:
# rw_sentence_pairs, __ = pair_generator(rw)

In [ ]:
id_pairs = get_pair_indices(vocab, rw_sentence_pairs)
id_pairs_1 = get_pair_indices(vocab, rw_sentence_pairs_)
id_pairs_2 = get_pair_indices(vocab, rrw_sentence_pairs_)
# id_pairs = get_pair_indices(vocab, sen_p1)
# id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [ ]:
id_pairs

In [ ]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

In [ ]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_1]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_1]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_1 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_1.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

In [ ]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_2]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_2]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_2 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_2.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

In [ ]:
prediction

In [ ]:
soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
soft.tolist()
soft_1 = torch.log_softmax(prediction_1, dim=1).argmax(dim=1)
soft_1.tolist()
soft_2 = torch.log_softmax(prediction_2, dim=1).argmax(dim=1)
soft_2.tolist()

In [ ]:
print(soft, soft_1, soft_2)

In [ ]:
ctr = 0
for i in range(len(soft)):
    if soft[i] == 2:
        print(rw_sentence_pairs[i])
        ctr = ctr + 1
print(ctr)

In [ ]:
ctr_1 = 0
for i in range(len(soft_1)):
    if soft_1[i] == 2:
        print(rw_sentence_pairs_[i])
        ctr_1 = ctr_1 + 1
print(ctr_1)

In [ ]:
ctr_2 = 0
for i in range(len(soft_2)):
    if soft_2[i] == 0:
        print(rrw_sentence_pairs_[i])
        ctr_2 = ctr_2 + 1
print(ctr_2)

In [ ]:
# id_pairs = get_pair_indices(vocab, rrw_sentence_pairs)
# # id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [ ]:
# premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
# hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

# premise_len    = list(map(len, premise_seq))
# hypothesis_len = list(map(len, hypothesis_seq))

# batch = len(premise_seq)
# temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
# premise_seq    = temp[:batch, :]
# hypothesis_seq = temp[batch:, :]

# prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# # prediction = prediction[prediction!=prediction[0,3]]

# prediction_list = prediction.to('cpu')
# prediction_list = prediction_list.tolist()
# print(len(prediction_list))

In [ ]:
# prediction

In [ ]:
# soft2 = torch.log_softmax(prediction, dim=1).argmax(dim=1)
# soft2.tolist()

In [ ]:
# ctr2 = 0
# for i in range(len(soft)):
#     if soft2[i] == 0:
#         print(rrw_sentence_pairs[i])
#         ctr2 = ctr2 + 1
# print(ctr2)

In [ ]:
e_100 = 0   # perfectly entailed pairs
e_50 = 0    # entailment
n_100 = 0   # perfectly neutral pairs
n_50 = 0    # neutral
cc = 0      # contradicting pairs
for i in range(len(soft_1)):
    if soft_1[i] == soft_2[i] == 0:
        e_100 = e_100 + 1
    elif (soft_1[i] == 0 and soft_2[i] == 1) or (soft_1[i] == 1 and soft_2[i] == 0) or (soft_1[i] == 0 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 0):
        e_50 = e_50 + 1
    elif soft_1[i] == soft_2[i] == 1:
        n_100 = n_100 + 1
    elif (soft_1[i] == 1 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 1):
        n_50 = n_50 + 1
    else: 
        cc = cc + 1
print('e100: ', e_100)
print('e50: ', e_50)
print('n100: ', n_100)
print('n50: ', n_50)
print('c100: ', cc)

In [ ]:
rw_sentence_pairs[0]

In [ ]:
prediction[0]

In [ ]:
rw_sentence_pairs[1]
# 0: entailment
# 1: neutral
# 2: contradiction

In [ ]:
maxi = torch.max(prediction, dim=0)
mini = torch.min(prediction, dim=0)
# print(maxi[0][0])
denom1 = float(maxi[0][0].to('cpu')) - float(mini[0][0].to('cpu'))
denom2 = float(maxi[0][1].to('cpu')) - float(mini[0][1].to('cpu'))
denom3 = float(maxi[0][2].to('cpu')) - float(mini[0][2].to('cpu'))
denom = ( denom1 + denom2 + denom3 ) / 3
print(maxi, mini)
print(denom)

In [ ]:
aa = []
for i in prediction:
    a = []
    for j in i:
        a.append(float(j.to('cpu'))/denom)
    aa.append(a)
print(aa)

In [ ]:
e_scores = []
for i in aa:
    e_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(e_scores)

In [ ]:
E_labels = []
for i in e_scores:
    if i < 0:
        E_labels.append('NEGATIVE ENTAILMENT')
    elif i > 0.3:
        E_labels.append('PERFECT ENTAILMENT')
    else:
        E_labels.append('NEUTRAL ENTAILMENT')

In [ ]:
ctrr = 0
for i in e_scores:
    if i < 0:
        ctrr = ctrr + 1
print(ctrr)

In [ ]:
df_final_E = pd.DataFrame(list(zip(rw_sentence_pairs, aa, e_scores, E_labels)), columns =['sentence pairs', 'entailment scores (raw: e, n, c)', 'entailment_metric [-1:1]', 'entailment_labels'])
df_final_E.to_csv('scores/summary_entailment_scores_of_test_pairs.csv')
df_final_E.head()

In [ ]:
maxi_1 = torch.max(prediction_1, dim=0)
mini_1 = torch.min(prediction_1, dim=0)
# print(maxi[0][0])
denom1_1 = float(maxi_1[0][0].to('cpu')) - float(mini_1[0][0].to('cpu'))
denom2_1 = float(maxi_1[0][1].to('cpu')) - float(mini_1[0][1].to('cpu'))
denom3_1 = float(maxi_1[0][2].to('cpu')) - float(mini_1[0][2].to('cpu'))
denom_1 = ( denom1_1 + denom2_1 + denom3_1 ) / 3
print(maxi_1, mini_1)
print(denom_1)

maxi_2 = torch.max(prediction_2, dim=0)
mini_2 = torch.min(prediction_2, dim=0)
# print(maxi[0][0])
denom1_2 = float(maxi_2[0][0].to('cpu')) - float(mini_2[0][0].to('cpu'))
denom2_2 = float(maxi_2[0][1].to('cpu')) - float(mini_2[0][1].to('cpu'))
denom3_2 = float(maxi_2[0][2].to('cpu')) - float(mini_2[0][2].to('cpu'))
denom_2 = ( denom1 + denom2 + denom3 ) / 3
print(maxi_2, mini_2)
print(denom_2)

In [ ]:
p1 = []
for i in prediction_1:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_1)
    p1.append(p)
print(p1)

p2 = []
for i in prediction_2:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_2)
    p2.append(p)
print(p2)

In [ ]:
me1_scores = []
for i in p1:
    me1_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me1_scores)

me2_scores = []
for i in p2:
    me2_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me2_scores)

In [ ]:
mutual_scores= []
for i, j in zip(me1_scores, me2_scores):
    mutual_scores.append( (i + j) / 2 )

In [ ]:
MUTUAL_labels = []
n_c = 0
p_c = 0
c_c = 0
for i in mutual_scores:
    if i <= -0.2:
        c_c += 1
        MUTUAL_labels.append('CONTRADICTION')
    elif i > -0.2 and i < 0:
        n_c += 1
        MUTUAL_labels.append('NEGATIVE ENTAILMENT')
    else:
        p_c += 1
        MUTUAL_labels.append('POSITIVE ENTAILMENT')
        
print("POSITIVE Entailments: ", p_c)
print("NEGATIVE Entailments: ", n_c)
print("Contradictions: ", c_c)

In [ ]:
df_final_ME = pd.DataFrame(list(zip(rw_sentence_pairs, p1, p2, me1_scores, me2_scores, mutual_scores, MUTUAL_labels)), columns =['sentence pairs', 'e_n_c scores 1', 'e_n_c scores 2', 'entailment_metric_1[-1:1]', 'entailment_metric_2[-1:1]', 'mutuality_score', 'mutual_labels'])
df_final_ME.to_csv('scores/mutual_entailment_scores_of_test_pairs.csv')
df_final_ME.head()